In [34]:
import numpy as np
import pandas as pd
import requests


from tqdm.notebook import tqdm_notebook
from tqdm import tqdm
tqdm.pandas()

import matplotlib
from matplotlib.pylab import *
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from mpl_toolkits.axes_grid1 import host_subplot
%matplotlib inline 

from datetime import datetime
import math 

pd.set_option('display.max_columns', None)  
pd.set_option('display.max_rows', None) 
pd.set_option('display.max_colwidth', None)
pd.options.display.float_format = '{:,.7f}'.format

### Import dataset

1. Import all cleaned data files 

- **aggregated_exchange_rate_vote_txs_DF** : Contains averaged oracle prices as provided by validators during per block 
- **market_swap_txs_DF** : Contains mainnet txs involving Luna <> UST swaps via Terra's Market module
- **astroport_ust_luna_txs_DF** : Contains mainnet txs involving Luna <> UST swaps via Astroport
- **aggregated_market_swap_txs_DF** : Contains total UST / LUNA which got minted / burnt per block


In [35]:
# Get DataFrames from pre-processed .csv files

## Oracle prices dataset
aggregated_exchange_rate_vote_txs_DF = pd.read_csv("./terra_classic_dataset/aggregated_exchange_rate_vote_txs_DF.csv") 

## LUNA <> UST Swaps via Astroport dataset
astroport_ust_luna_txs_DF = pd.read_csv("./terra_classic_dataset/astroport_ust_luna_txs_DF.csv") 

## LUNA <> UST Market Module swaps which involve mint / burn
market_swap_txs_DF = pd.read_csv("./terra_classic_dataset/market_swap_txs_DF.csv") 

## Market Swaps via Module :: Aggregated Trading Volumes / Supply contraction / expansion dynamics
aggregated_market_swap_txs_DF = pd.read_csv("./terra_classic_dataset/aggregated_market_swap_txs_DF.csv") 


# Remove index column
aggregated_exchange_rate_vote_txs_DF.drop(['Unnamed: 0'],axis=1,inplace=True)
astroport_ust_luna_txs_DF.drop(['Unnamed: 0'],axis=1,inplace=True)
aggregated_market_swap_txs_DF.drop(['Unnamed: 0'],axis=1,inplace=True)
market_swap_txs_DF.drop(['Unnamed: 0'],axis=1,inplace=True)




**Data Cleaning :: Step 1 - Filter data for only common blocks**

In [36]:
# Get Block heights for which data is available and block height range for which simulation is to be executed

astroport_txs_DF_start_block = astroport_ust_luna_txs_DF.iloc[0]["BlockHeight"]
astroport_txs_DF_end_block = astroport_ust_luna_txs_DF.iloc[len(astroport_ust_luna_txs_DF.index) - 1]["BlockHeight"]
print(f"astroport_txs_DF || Start block = {astroport_txs_DF_start_block}  End block = {astroport_txs_DF_end_block} \
Total Blocks = {astroport_txs_DF_end_block - astroport_txs_DF_start_block}")

aggregated_market_swap_txs_DF_start_block = aggregated_market_swap_txs_DF.iloc[0]["BlockHeight"]
aggregated_market_swap_txs_DF_end_block = aggregated_market_swap_txs_DF.iloc[len(aggregated_market_swap_txs_DF.index) - 1]["BlockHeight"]
print(f"aggregated_market_swap_txs_DF || Start block = {aggregated_market_swap_txs_DF_start_block}  End block = {aggregated_market_swap_txs_DF_end_block} \
Total Blocks = {aggregated_market_swap_txs_DF_end_block - aggregated_market_swap_txs_DF_start_block}")

market_swap_txs_DF_start_block = market_swap_txs_DF.iloc[0]["BlockHeight"]
market_swap_txs_DF_end_block = market_swap_txs_DF.iloc[len(market_swap_txs_DF.index) - 1]["BlockHeight"]
print(f"market_swap_txs_DF || Start block = {market_swap_txs_DF_start_block}  End block = {market_swap_txs_DF_end_block} \
Total Blocks = {market_swap_txs_DF_end_block - market_swap_txs_DF_start_block}")

oracle_txs_DF_start_block = aggregated_exchange_rate_vote_txs_DF.iloc[0]["BlockHeight"]
oracle_txs_DF_end_block = aggregated_exchange_rate_vote_txs_DF.iloc[len(aggregated_exchange_rate_vote_txs_DF.index) - 1]["BlockHeight"]
print(f"oracle_txs_DF || Start block = {oracle_txs_DF_start_block}  End block = {oracle_txs_DF_end_block} \
Total Blocks = {oracle_txs_DF_end_block - oracle_txs_DF_start_block}")

START_BLOCK = int(max(astroport_txs_DF_start_block, max(market_swap_txs_DF_start_block, oracle_txs_DF_start_block)))
START_BLOCK = int(max(START_BLOCK, aggregated_market_swap_txs_DF_start_block))
END_BLOCK = int(min(astroport_txs_DF_end_block, min(market_swap_txs_DF_end_block, oracle_txs_DF_end_block)))
END_BLOCK = int(min(END_BLOCK, aggregated_market_swap_txs_DF_end_block))
print(f"\nCommon Range, Start block = {START_BLOCK} End block = {END_BLOCK} | Total blocks = {END_BLOCK - START_BLOCK}")



astroport_txs_DF || Start block = 7117990  End block = 7622720 Total Blocks = 504730
aggregated_market_swap_txs_DF || Start block = 6960461.0  End block = 7595342.0 Total Blocks = 634881.0
market_swap_txs_DF || Start block = 6960461  End block = 7608047 Total Blocks = 647586
oracle_txs_DF || Start block = 6958375.0  End block = 7608054.0 Total Blocks = 649679.0

Common Range, Start block = 7117990 End block = 7595342 | Total blocks = 477352


In [37]:
import datetime
import calendar

# START BLOCK : 2022.04.04 19:25:06+04:00   -   https://finder.terra.money/classic/blocks/7117990
start_block_time = datetime.datetime(2022, 4, 4, 19, 25, 6)
ttuple = start_block_time.timetuple()
start_block_timestamp = calendar.timegm(ttuple)

# END BLOCK : 2022.05.13 16:03:06+04:00   -   https://finder.terra.money/classic/blocks/7595342
end_block_time = datetime.datetime(2022, 5, 13, 16, 3, 6)
ttuple = end_block_time.timetuple()
end_block_timestamp = calendar.timegm(ttuple)

difference_timestamps = end_block_timestamp - start_block_timestamp
average_block_time = difference_timestamps / (END_BLOCK - START_BLOCK)

print(f"Start block time : {start_block_time} | Timestamp = {start_block_timestamp}")
print(f"End block time : {end_block_time} | Timestamp = {end_block_timestamp}")
print(f"Average block time = {average_block_time}")



Start block time : 2022-04-04 19:25:06 | Timestamp = 1649100306
End block time : 2022-05-13 16:03:06 | Timestamp = 1652457786
Average block time = 7.033551760545677


In [38]:
# Shorten dataset for faster execution by taking only dataset for only common blocks
aggregated_exchange_rate_vote_txs_DF = aggregated_exchange_rate_vote_txs_DF.loc[(aggregated_exchange_rate_vote_txs_DF['BlockHeight'] >= START_BLOCK) & (aggregated_exchange_rate_vote_txs_DF['BlockHeight'] <= END_BLOCK)]
astroport_ust_luna_txs_DF = astroport_ust_luna_txs_DF.loc[(astroport_ust_luna_txs_DF['BlockHeight'] >= START_BLOCK) & (astroport_ust_luna_txs_DF['BlockHeight'] <= END_BLOCK)]
aggregated_market_swap_txs_DF = aggregated_market_swap_txs_DF.loc[(aggregated_market_swap_txs_DF['BlockHeight'] >= START_BLOCK) & (aggregated_market_swap_txs_DF['BlockHeight'] <= END_BLOCK)]
market_swap_txs_DF = market_swap_txs_DF.loc[(market_swap_txs_DF['BlockHeight'] >= START_BLOCK) & (market_swap_txs_DF['BlockHeight'] <= END_BLOCK)]



**crash_analysis_DF : DataFrame containing cleaned summarized data for each block**

- Replace 0 values with last valid provided oracle prices
- Create new DF by merging available DFs on BlockHeight
- Fill in na values for astroport prices for missing blocks with last valid values
- Drop duplicate rows based on BlockHeight value
- Calculate UST and LUNA exchange rate in USD terms

In [39]:
# Fill in missing Oracle prices values based on previous block inputs
aggregated_exchange_rate_vote_txs_DF.replace(to_replace = 0,  method='bfill', inplace=True)

# Merge DFs and select columns we need 
crash_analysis_DF = pd.merge(aggregated_exchange_rate_vote_txs_DF,aggregated_market_swap_txs_DF, on='BlockHeight',how='left')
crash_analysis_DF = pd.merge(crash_analysis_DF, astroport_ust_luna_txs_DF, on='BlockHeight',how='left')
crash_analysis_DF = crash_analysis_DF[["BlockHeight","uusd::minted","uusd::burnt","uluna::minted","uluna::burnt","uusd::mean","Price (luna/ust)","Price (ust/luna)"]]

# Fill in missing values via forward propoagating correct values
crash_analysis_DF["Price (luna/ust)"].fillna(method="ffill",inplace=True)
crash_analysis_DF["Price (ust/luna)"].fillna(method="ffill",inplace=True)

# Remove duplicates based on block height
crash_analysis_DF.drop_duplicates(subset=["BlockHeight"],inplace=True )

# Calculate UST's price in USD based on Astroport and Oracle data available
crash_analysis_DF["ExchangeRate::USD/UST"] = crash_analysis_DF.progress_apply(lambda x: x["uusd::mean"]*x["Price (luna/ust)"] , axis=1)

# Rename columns
crash_analysis_DF.rename(columns={"uusd::mean":"ExchangeRate::USD/LUNA","Price (luna/ust)":"ExchangeRate::LUNA/UST","Price (ust/luna)":"ExchangeRate::UST/LUNA"}, inplace=True)


100%|██████████| 477353/477353 [00:02<00:00, 167341.52it/s]


In [40]:
crash_analysis_DF.head(4)

,BlockHeight,uusd::minted,uusd::burnt,uluna::minted,uluna::burnt,ExchangeRate::USD/LUNA,ExchangeRate::LUNA/UST,ExchangeRate::UST/LUNA,ExchangeRate::USD/UST
0,7117990,"10,434.7495680",0.0000000,0.0000000,94.2895450,110.7717770,0.0089910,111.2217760,0.9959540
1,7117991,0.0000000,0.0000000,0.0000000,0.0000000,110.7781370,0.0089908,111.2253395,0.9959793
2,7117992,0.0000000,0.0000000,0.0000000,0.0000000,110.7115950,0.0089908,111.2253395,0.9953810
3,7117993,"11,444.3323410",0.0000000,0.0000000,103.4122460,110.7115950,0.0089908,111.2250797,0.9953834


In [41]:
# Create new columns for LUNA / UST Supply values
crash_analysis_DF['LUNA Supply'] = 0
crash_analysis_DF['UST Supply'] = 0


In [42]:
# Enter initial LUNA / UST Supply values, and timestamp
crash_analysis_DF.loc[0, 'LUNA Supply'] = 800_000_000     # 800 Million 
crash_analysis_DF.loc[0, 'UST Supply'] = 15_000_000_000   # 15 Billion

# We calculate estimated timestamp for a certain block, assuming block time ~ 7 sec
crash_analysis_DF.loc[0, 'timestamp'] = start_block_timestamp 

# Reset Index
crash_analysis_DF.reset_index(inplace=True)
crash_analysis_DF.drop('index', axis=1, inplace=True)

# Calculate new LUNA / UST total Supplies for each block
for i in tqdm_notebook(range(1, len(crash_analysis_DF))):
    crash_analysis_DF.loc[i, 'LUNA Supply'] = crash_analysis_DF.loc[i-1, 'LUNA Supply'] + crash_analysis_DF.loc[i, 'uluna::minted'] - crash_analysis_DF.loc[i, 'uluna::burnt']
    crash_analysis_DF.loc[i, 'UST Supply'] = crash_analysis_DF.loc[i-1, 'UST Supply'] + crash_analysis_DF.loc[i, 'uusd::minted'] - crash_analysis_DF.loc[i, 'uusd::burnt']
    crash_analysis_DF.loc[i, 'timestamp'] = crash_analysis_DF.loc[i-1, 'timestamp'] + average_block_time


# Calculate dateTime
crash_analysis_DF["dateTime"] = crash_analysis_DF.progress_apply(lambda x: datetime.datetime.fromtimestamp(int(x["timestamp"])) , axis=1)
crash_analysis_DF["timestamp"] = crash_analysis_DF.progress_apply(lambda x: int(x["timestamp"]) , axis=1)


  0%|          | 0/477352 [00:00<?, ?it/s]

100%|██████████| 477353/477353 [00:02<00:00, 187078.93it/s]


**Set initial LUNA supply and UST supply and calculate the total supplies for each block**

In [44]:
astroport_ust_luna_txs_DF.head(4)


,BlockHeight,offer_asset,offer_amount,ask_asset,return_amount,spread_amount,belief_price,Price (luna/ust),Price (ust/luna)
0,7117990,uusd,31.7352430,uluna,0.2853330,0.0000010,111.2100833,0.0089910,111.2217760
1,7117991,uusd,"5,000.0000000",uluna,44.9537850,0.0014930,111.2252925,0.0089908,111.2253395
2,7117993,uusd,"1,000.0000000",uluna,8.9907780,0.0000600,111.2223953,0.0089908,111.2250797
3,7117993,uusd,150.0000000,uluna,1.3485680,0.0000010,111.2217562,0.0089905,111.2290963


In [33]:
cur_astroport_swap_txs_DF = astroport_ust_luna_txs_DF.loc[ astroport_ust_luna_txs_DF["BlockHeight"] == 7118065 ]
cur_astroport_swap_txs_DF

,BlockHeight,offer_asset,offer_amount,ask_asset,return_amount,spread_amount,belief_price,Price (luna/ust),Price (ust/luna)
38,7118065,uluna,26.5727980,uusd,"2,939.7383640",0.0575450,0.0090435,0.0090392,110.6296132


# TERRA CLASSIC : Possible path to recovery ?

Here we simulate our proposed model's performance by replaying market swap and swaps on astroport via our updated market module and astroport's pool logic. 

**We have made the following updates made to the Market Module and astroport's native coins pool**

- Astroport's LUNA <> UST pool checks for arb opportunity between itself and the market module during every tx, and executes the arb if the opportunity exits. 
    - Purpose : 
        - Terra classic is most prone to centralizaton and control in-case market the burn / mint logic is re-enabled by Validators / MEV searchers. 
        - Allowing the stable-coins arbitrage to happen internally during astroport swaps negates the potential value extraction possible by MEV searchers and distributes the same among the community.
        - Interal Arbitrage in astroport pools will also prevent any pool imbalance possibilites and will allow for smoother price discovery for terra coins. 

<br>
        
- Market Module queries a oracle contract which computes TWAP price for terra-coins denominated in LUNA as they are traded on astroport. 
    - Purpose : 
        - One of the post prone issues associated with terra classic is the underlying assumption of terra coins being always pegged to their denominted currencies. We don't believe that the promise of terra coins being traded at their pegged prices can be met by the chain during market downturns, so the mechanism needs to be altered from being stable-coins to dynamic-stability mechanism, where the terra-coin is assumed to be at a discount during harsh market conditions, so that the chain can absort the sell / burn pressure at competitive spreads without compromising its security guarantees via LUNA inflation. 
        - We are of the opinion that Astroport's pools are the most credible source for computing exchange price between LUNA and Terra coins via TWAP. Reasoning being that the astroport pools are the biggest pools which facilitaite off-chain price discovery for terra coins, and with the internal arbitrage modification introduced in the point above, they are most prone to any sort of oracle exploits. 

<br>


- Market Module uses the provided oracle prices, the TWAP price from astroport along with a variable drift parameter to calculate the exchange price between terra coins and LUNC. 
    - The purpose of the drift parameter is to try to bring terra stables closer to their target pegs over time.

<br>

- LUNC burned via the 1.2% tax is used to keep track of the maximum amount of LUNC which can be minted by the market module by burning Terra coins. This max mintable LUNC amount undergoes regression every day reducing the amount, implying that LUNC still inherits its deflationary characterisitc while still being able to support the terra coins with their peg in a "limited" manner.
   



In [372]:
# Initial Parameters
init_basepool = 50_000_000   # 50 M SDR
init_minstabilityspread = 0.05    # 5%
init_poolrecoveryperiod = 36      # regression parameter



In [373]:
crash_analysis_DF.head(3)

,BlockHeight,uusd::minted,uusd::burnt,uluna::minted,uluna::burnt,ExchangeRate::USD/LUNA,ExchangeRate::LUNA/UST,ExchangeRate::UST/LUNA,ExchangeRate::USD/UST,LUNA Supply,UST Supply,timestamp,dateTime
0,7117990,"10,434.7495680",0.0000000,0.0000000,94.2895450,110.7717770,0.0089910,111.2217760,0.9959540,"800,000,000.0000000","15,000,000,000.0000000",1649100306,2022-04-04 23:25:06
1,7117991,0.0000000,0.0000000,0.0000000,0.0000000,110.7781370,0.0089908,111.2253395,0.9959793,"800,000,000.0000000","15,000,000,000.0000000",1649100313,2022-04-04 23:25:13
2,7117992,0.0000000,0.0000000,0.0000000,0.0000000,110.7115950,0.0089908,111.2253395,0.9953810,"800,000,000.0000000","15,000,000,000.0000000",1649100320,2022-04-04 23:25:20


In [374]:

def simulate_terra_classic(terra_classic_instance, market_swap_txs_DF, astroport_ust_luna_txs_DF, crash_analysis_DF, oracle_exchange_rate_DF, start_height, end_height):
        
    # DataFrame to store simulated swaps metrics
    simulation_metrics_DF = pd.DataFrame(columns=['BlockHeight','burnt_denom', 'tokens_burnt', 'minted_denom',
       'tokens_minted', 'swap_amount', 'swap_fee_amount','spread', 'terraPool','lunaPool', 'delta','ukrw_oracle_price','usdr_oracle_price'\
                                                 ,'ust_oracle_price','usd_per_ust_price','usd_per_luna_price', 'expected_profit'])
    
    # DataFrame to store metrics around burning $100k worth UST for LUNA in each block
    simulated_ust_burn_metrics_DF = pd.DataFrame(columns=['BlockHeight', 'ust_burnt', 'uluna_minted' ,'spread','delta', 'lunaPool','terra_pool','usd_per_luna_price', 'usd_per_ust_price'])

    # DataFrame to store metrics around burning $100k worth LUNA for UST in each block
    simulated_uluna_burn_metrics_DF = pd.DataFrame(columns=['BlockHeight', 'uluna_burnt','ust_minted','spread','delta', 'lunaPool','terra_pool','usd_per_luna_price', 'usd_per_ust_price'])
    
    # DataFrame to store metrics around each Astroport Swap Tx 
    simulated_astroport_pool_swap_txs_DF = pd.DataFrame(columns=['BlockHeight', 'offer_token','offer_amount','return_token','return_amount', 'arbitrage_profit_denom' , 'arbitrage_profit_amount' ,'TWAP Price (ust)', 'TWAP Price (uluna)'])
    
    

    cur_height = start_height
    prices = {"ukrw": 0, "usdr": 0, "ust": 0}    
    price_till = 0
    
    # Process all blocks 
    for cur_height in tqdm_notebook(range(start_height,end_height)):
        
        
        
        # Oracle Prices
        ukrw_oracle_price = terra_classic_instance.OracleKeeper.GetLunaExchangeRate("ukrw")
        usdr_oracle_price = terra_classic_instance.OracleKeeper.GetLunaExchangeRate("usdr")
        ust_oracle_price = terra_classic_instance.OracleKeeper.GetLunaExchangeRate("ust")
        
        # Get UST and LUNA's actual USD Price ()
        crash_analysis_DF_cur_block = crash_analysis_DF.loc[ crash_analysis_DF["BlockHeight"] == cur_height ]        
        ust_price_in_usd = crash_analysis_DF_cur_block.iloc[0]["ExchangeRate::USD/UST"]
        luna_price_in_usd = crash_analysis_DF_cur_block.iloc[0]["ExchangeRate::USD/LUNA"]
        
        # Get current timestamp
        current_timestamp = int(crash_analysis_DF_cur_block.iloc[0]["timestamp"])
        
        #------------------------------------------------------
        #--------------- MARKET SWAP TXS ----------------------
        #------------------------------------------------------
                            
        # Get all market swap txs to be processed for the current block
        cur_swap_txs_DF = market_swap_txs_DF.loc[ market_swap_txs_DF["BlockHeight"] == cur_height ]
        
        # Loop over all txs for each block
        for tx_row in cur_swap_txs_DF.to_dict(orient='records'):
            try:
                # Process swap tx :: Only UST burn txs are considered
                if tx_row["burnt_denom"] == "ust" and tx_row["minted_denom"] == "uluna": 
                    # print(f"\nblock {cur_height} :: Burn {tx_row['burnt_denom']} {tx_row['tokens_burnt']} tokens and mint {tx_row['minted_denom']} tokens")
                    swap_response = terra_classic_instance.Swap("trader", {"denom":tx_row["burnt_denom"], "amount":float(tx_row["tokens_burnt"]) }, tx_row["minted_denom"])
                    swapCoin = swap_response["SwapCoin"] 
                    swapFee = swap_response["SwapFee"] 
                    spread = swap_response["spread"] 
                    err = swap_response["Error"] 

                    if err != None:
                        print(err)
                    else:
                        # Query market module params
                        terraPool = terra_classic_instance.get_TerraPool()
                        lunaPool = terra_classic_instance.get_LunaPool_fromTerraPool()
                        delta = terra_classic_instance.GetTerraPoolDelta()
                        
                        expected_profit = 0
                        if tx_row["burnt_denom"] == "ust": # If UST is burnt and LUNA is minted
                            expected_profit = (float(swapCoin["amount"]) * luna_price_in_usd) -  (tx_row['tokens_burnt'] * ust_price_in_usd)
                        else:  # If LUNA is burnt and UST is minted
                            expected_profit = (float(swapCoin["amount"]) * ust_price_in_usd) -  (tx_row['tokens_burnt'] * luna_price_in_usd)
                        
                        try:
                            # store metrics
                            simulation_metrics_DF.loc[len(simulation_metrics_DF.index)] = [cur_height, tx_row["burnt_denom"],\
                                                        tx_row["tokens_burnt"], tx_row["minted_denom"], float(swapCoin["amount"]) + float(swapFee["amount"]),\
                                                        swapCoin["amount"], swapFee["amount"], spread, terraPool, lunaPool, delta, ukrw_oracle_price[0],\
                                                        usdr_oracle_price[0], ust_oracle_price[0], ust_price_in_usd, luna_price_in_usd, expected_profit ]
#                             print(f"simulation_metrics_DF Length = {len(simulation_metrics_DF.index)}")
                        except Exception as e:    
                            print(e)
            except Exception as e:
                print(f"\nblock {cur_height} :: Burn {tx_row['burnt_denom']} {tx_row['tokens_burnt']} tokens and mint {tx_row['minted_denom']} tokens")
                print(e)
                
                
        #------------------------------------------------------
        #--------------- ASTROPORT SWAP TXS -------------------
        #------------------------------------------------------

        # Get all market swap txs to be processed for the current block
        cur_astroport_swap_txs_DF = astroport_ust_luna_txs_DF.loc[ astroport_ust_luna_txs_DF["BlockHeight"] == cur_height ]
                                
        # Loop over all txs for each block
        for tx_row in cur_astroport_swap_txs_DF.to_dict(orient='records'):
            try:
                # Process Astroport swap tx 
                astroport_swap_response = terra_classic_instance.astroport_swap({"denom": tx_row["offer_asset"] , "amount": float(tx_row["offer_amount"]) }, float(tx_row["belief_price"]), 0)
                if astroport_swap_response["error"] != None:
                     print(err)
                else:       
                    twap_price_ust = terra_classic_instance.astroport_get_cumulative_price("ust")
                    twap_price_uluna = terra_classic_instance.astroport_get_cumulative_price("uluna")
                    try:
                        simulated_astroport_pool_swap_txs_DF[len(simulated_astroport_pool_swap_txs_DF.index)] = [cur_height,\
                                                            tx_row["offer_asset"], tx_row["offer_amount"], astroport_swap_response["return_asset"]["denom"],\
                                                            astroport_swap_response["return_asset"]["amount"], astroport_swap_response["arb_response"]["denom"]\
                                                            , astroport_swap_response["arb_response"]["amount"], twap_price_uusd, twap_price_uluna ]
                    except Exception as e:    
                        print(e)
            except Exception as e:
                print(f"\nblock {cur_height} :: Burn {tx_row['burnt_denom']} {tx_row['tokens_burnt']} tokens and mint {tx_row['minted_denom']} tokens")
                print(e)                
        
        #-----------------------------------------------
        #--------------- UPDATE TWAP -------------------
        #-----------------------------------------------
        
        terra_classic_instance.record_twap_snapshots(current_timestamp)
                
            
            
            
        # Query market module params
        terraPool = terra_classic_instance.get_TerraPool()
        lunaPool = terra_classic_instance.get_LunaPool_fromTerraPool()
        delta = terra_classic_instance.GetTerraPoolDelta()            
        
        # DataFrame to store metrics around burning $100k worth UST for LUNA in each block
        ustc_tokens_to_burn = int(100_000 / float(ust_price_in_usd))
        ust_burn_res = terra_classic_instance.simulateSwap({"denom":"ust","amount": ustc_tokens_to_burn }, "uluna")
        ust_burn_res_returnCoin = ust_burn_res["return_denom"] 
        ust_burn_res_spread = ust_burn_res["spread"] 
        ust_burn_res_err = ust_burn_res["error"]     
        if ust_burn_res_err!= None:
            print(f"UST Burn Simulation Error : {ust_burn_res_err}")
        else:
            simulated_ust_burn_metrics_DF.loc[len(simulated_ust_burn_metrics_DF.index)] =  [cur_height, ustc_tokens_to_burn, ust_burn_res_returnCoin["amount"],ust_burn_res_spread,delta, lunaPool, terraPool,luna_price_in_usd, ust_price_in_usd]

            
        # DataFrame to store metrics around burning $100k worth LUNA for UST in each block
        uluna_tokens_to_burn = int(100_000 / float(luna_price_in_usd))
        uluna_burn_res = terra_classic_instance.simulateSwap({"denom":"uluna","amount": uluna_tokens_to_burn }, "ust")
        uluna_burn_res_returnCoin = uluna_burn_res["return_denom"] 
        uluna_burn_res_spread = uluna_burn_res["spread"] 
        uluna_burn_res_err = uluna_burn_res["error"]     
        if uluna_burn_res_err!= None:
            print(f"ULUNA Burn Simulation Error : {uluna_burn_res_err}")
        else:
            simulated_uluna_burn_metrics_DF.loc[len(simulated_uluna_burn_metrics_DF.index)] =  [cur_height, uluna_tokens_to_burn, uluna_burn_res_returnCoin["amount"],uluna_burn_res_spread,delta, lunaPool, terraPool,luna_price_in_usd, ust_price_in_usd]

               
        # Replenish Market module's pool after each block
        terra_classic_instance.TerraMarketModuleEndBlock()
                            
                        
        # Update oracle prices every 5 blocks
        cur_price_DF = oracle_exchange_rate_DF.loc[ oracle_exchange_rate_DF["BlockHeight"] == cur_height ] 
    
        # Set oracle prices 
        terra_classic_instance.OracleKeeper.SetLunaExchangeRate("ukrw", cur_price_DF.iloc[0]["ukrw::mean"] )
#         print(f"{cur_height} || ukrw price = {cur_price_DF.iloc[0]['ukrw::mean']}")
        terra_classic_instance.OracleKeeper.SetLunaExchangeRate("usdr", cur_price_DF.iloc[0]["usdr::mean"] )
#         print(f"{cur_height} || usdr price = {cur_price_DF.iloc[0]['usdr::mean']}")
        terra_classic_instance.OracleKeeper.SetLunaExchangeRate("ust", cur_price_DF.iloc[0]["uusd::mean"] )
#         print(f"{cur_height} || ust price = {cur_price_DF.iloc[0]['uusd::mean']}")
                    
    
    return (simulation_metrics_DF, simulated_ust_burn_metrics_DF, simulated_uluna_burn_metrics_DF)






In [375]:
# CREATE TERRA CLASSIC INSTANCE 
terra_classic_instance = TerraMarketModule(init_basepool, init_minstabilityspread, init_poolrecoveryperiod)
terra_classic_instance.BankKeeper.MintCoins("trader",{"denom":"uluna", "amount":800 * 10**6 })
terra_classic_instance.BankKeeper.MintCoins("trader",{"denom":"ust", "amount":18000 * 10**6 })

# Set initial exchange rates
terra_classic_instance.OracleKeeper.SetLunaExchangeRate("ust", 3.074 )
terra_classic_instance.OracleKeeper.SetLunaExchangeRate("usdr", 2.290 )
terra_classic_instance.OracleKeeper.SetLunaExchangeRate("ukrw", 3916.569 )



NameError: name 'TerraMarketModule' is not defined

In [376]:
# # ------------------------------------------------------------
# # START BLOCK : 2022.04.04 19:25:31 + 04:00   -   https://finder.terra.money/classic/blocks/7117994
# start_block_time_swaps = datetime.datetime(2022, 4, 4, 19, 25, 31)
# ttuple = start_block_time_swaps.timetuple()
# start_block_timestamp = calendar.timegm(ttuple)

# # Calculate timestamp for simulation_metrics_DF 
# simulation_metrics_DF["timestamp"] = 0
# simulation_metrics_DF.loc[0, 'timestamp'] = start_block_timestamp
# for i in tqdm_notebook(range(1, len(simulation_metrics_DF))):
#     new_timestamp = int(simulation_metrics_DF.loc[i-1, 'timestamp']) + ((int(simulation_metrics_DF.loc[i, 'BlockHeight']) - int(simulation_metrics_DF.loc[i-1, 'BlockHeight']))*average_block_time )
#     new_timestamp = int(new_timestamp)
#     simulation_metrics_DF.loc[i, 'timestamp'] = int(new_timestamp)

# # Calculate dateTime for simulation_metrics_DF
# simulation_metrics_DF["dateTime"] = simulation_metrics_DF.progress_apply(lambda x: datetime.datetime.fromtimestamp(int(x["timestamp"])) , axis=1)
# # ------------------------------------------------------------    
# # ------------------------------------------------------------
# # START BLOCK : 2022.04.04 19:25:31 + 04:00   -   https://finder.terra.money/classic/blocks/7117990
# start_block_time_swaps = datetime.datetime(2022, 4, 4, 19, 25, 4)
# ttuple = start_block_time_swaps.timetuple()
# start_block_timestamp = calendar.timegm(ttuple)

# # Calculate timestamp for simulated_ust_burn_metrics_DF 
# simulated_ust_burn_metrics_DF["timestamp"] = 0
# simulated_ust_burn_metrics_DF.loc[0, 'timestamp'] = start_block_timestamp
# for i in tqdm_notebook(range(1, len(simulated_ust_burn_metrics_DF))):
#     new_timestamp = simulated_ust_burn_metrics_DF.loc[i-1, 'timestamp'] + ((int(simulated_ust_burn_metrics_DF.loc[i, 'BlockHeight']) - int(simulated_ust_burn_metrics_DF.loc[i-1, 'BlockHeight']))*average_block_time )
#     new_timestamp = int(new_timestamp)
#     simulated_ust_burn_metrics_DF.loc[i, 'timestamp'] = int(new_timestamp)
# #     print(f"{simulated_ust_burn_metrics_DF.loc[i, 'BlockHeight']} -- {simulated_ust_burn_metrics_DF.loc[i, 'timestamp']}")

# # Calculate dateTime for simulated_ust_burn_metrics_DF
# simulated_ust_burn_metrics_DF["dateTime"] = simulated_ust_burn_metrics_DF.progress_apply(lambda x: datetime.datetime.fromtimestamp(int(x["timestamp"])) , axis=1)
# # ------------------------------------------------------------
# # ------------------------------------------------------------
# # START BLOCK : 2022.04.04 19:25:31 + 04:00   -   https://finder.terra.money/classic/blocks/7117990
# start_block_time_swaps = datetime.datetime(2022, 4, 4, 19, 25, 4)
# ttuple = start_block_time_swaps.timetuple()
# start_block_timestamp = calendar.timegm(ttuple)

# # Calculate timestamp for simulated_uluna_burn_metrics_DF 
# simulated_uluna_burn_metrics_DF["timestamp"] = 0
# simulated_uluna_burn_metrics_DF.loc[0, 'timestamp'] = start_block_timestamp
# for i in tqdm_notebook(range(1, len(simulated_uluna_burn_metrics_DF))):
#     prev_timestamp = simulated_uluna_burn_metrics_DF.loc[i-1, 'timestamp']
#     prev_blockheight = int(simulated_uluna_burn_metrics_DF.loc[i-1, 'BlockHeight'])
#     cur_blockheight = int(simulated_uluna_burn_metrics_DF.loc[i, 'BlockHeight'])
                        
#     new_timestamp = prev_timestamp + ((cur_blockheight - prev_blockheight)*average_block_time)
#     new_timestamp = int(new_timestamp)
# #     print(f"prev_blockheight={prev_blockheight} -- cur_blockheight = {cur_blockheight} || prev_timestamp={prev_timestamp} -- new_timestamp = {new_timestamp}")
                        
#     simulated_uluna_burn_metrics_DF.loc[i, 'timestamp'] = int(new_timestamp)

# # Calculate dateTime for simulated_uluna_burn_metrics_DF
# simulated_uluna_burn_metrics_DF["dateTime"] = simulated_uluna_burn_metrics_DF.progress_apply(lambda x: datetime.datetime.fromtimestamp(int(x["timestamp"])) , axis=1)
           

In [377]:
# simulated_ust_burn_metrics_DF["BlockHeight"] = simulated_ust_burn_metrics_DF.apply(lambda x : int(x["BlockHeight"]), axis=1)
# simulated_uluna_burn_metrics_DF["BlockHeight"] = simulated_uluna_burn_metrics_DF.apply(lambda x : int(x["BlockHeight"]), axis=1)



In [379]:
# SAVE Files 
# simul_name = "simulation_"
# simulation_metrics_DF.to_csv(simul_name + "simulation_metrics_DF.csv")
# simulated_ust_burn_metrics_DF.to_csv(simul_name + "simulated_ust_burn_metrics_DF.csv")
# simulated_uluna_burn_metrics_DF.to_csv(simul_name + "simulated_uluna_burn_metrics_DF.csv")

## ORACLE MODULE

A toned-down version of terra's Oracle module used to update / fetch prices for terraCoins to be used by Terra's market module for processing swaps.


In [380]:
# OracleKeeper defines expected oracle keeper
#---------------x-----------x--------------
class TerraOracleKeeper:

    def __new__(cls, *args, **kwargs):
        return super().__new__(cls)
    
    def __init__(self):    
        self.voteperiod = 30                       # core.BlocksPerMinute / 2 (30 seconds) 
        self.rewarddistributionwindow = 5256000    # core.BlocksPerWeek (window for a week)
        self.slashwindow = 100800                  # core.BlocksPerYear (window for a year)

        self.votethreshold = 0.500000000000000000  # 50%
        self.rewardband = 0.02                     # 2%
        
        self.whitelist = {"name": "ukrw", "tobin_tax": 0.002000000000000000}
        self.slashfraction = 0.001000000000000000                 # 0.01%
        self.minvalidperwindow = 0.050000000000000000             # 5%
        
        self.currentExchangeRates = {
            "ust": 0,
            "usdr": 0,
        }
        self.toblinTax = {
            "ust": 0.0035,
            "usdr": 0.0035,
        }
    
        self.MicroLunaDenom = "uluna"
        self.MicroUSDDenom  = "ust"
        self.MicroSDRDenom  = "usdr"
        
    # EXTERNAL FUNCTION
    # get exchange rate
    def GetLunaExchangeRate(self,denom):
        if denom == self.MicroLunaDenom:
            return 1, None
        # retrieve exchange rate
        ex = self.currentExchangeRates[denom]
        if ex == None:
            return 0, "Unknown Denom"    
        return ex, None

    # INTERNAL FUNCTION
    # set exchange rate
    def SetLunaExchangeRate(self, denom, exchangeRate): 
        self.currentExchangeRates[denom] = exchangeRate
       
    
    # INTERNAL FUNCTION
    # delete exchange rate
    def _DeleteLunaExchangeRate(self, denom): 
        self.currentExchangeRates[denom] = 0
           
    # EXTERNAL FUNCTION -- Get Toblin Tax
    def GetTobinTax(self, denom):
        return self.toblinTax[denom]

    # EXTERNAL FUNCTION -- Set Toblin Tax
    def SetTobinTax(self, denom, tobinTax): 
        self.toblinTax[denom] = tobinTax
   

# Bank Module 

Bank Module keeps track of user's token balances and the total supply of these tokens.

In [381]:
# BankKeeper defines expected supply keeper
#---------------x-----------x--------------
class TerraBankModule:
    
    
    def __new__(cls, *args, **kwargs):
        return super().__new__(cls)
    
    def __init__(self):
        self.accounts = pd.DataFrame(columns=["user_address","uluna","ust","usdr","ust_uluna_lp_token"])
        self.accounts.loc[0] = ["",0,0,0,0]
        self.totalSupply = {
        "uluna": 0,
        "ust": 0,
        "usdr": 0,
        "ust_uluna_lp_token":0
    }
        
    # TRANSFER TOKENS
    def SendCoinsFromModuleToModule(self, senderAddr, recipientAddr,  coin):
        sender = self.accounts.loc[self.accounts['user_address'] == senderAddr]
        if len(sender) == 0: # Return Error 
            return "Sender doesn't exist"

        # balance check
        sender_balance = float(self.accounts.loc[self.accounts['user_address'] == senderAddr][coin["denom"]])        
        if sender_balance < float(coin["amount"]):
            return "insufficient balance"        
        
        # Get recepient
        recepient = self.accounts.loc[self.accounts['user_address'] == recipientAddr]
        if len(recepient) == 0: # Create user 
            self.accounts.loc[len(self.accounts.index)] = [recipientAddr,0,0,0,0]
        recepient = self.accounts.loc[self.accounts['user_address'] == recipientAddr]
        
        # Update recepient user balance in the dataframe
        recepient_balance = float(recepient[coin['denom']])
        self.accounts.loc[self.accounts['user_address'] == recipientAddr, coin["denom"]] = recepient_balance + float(coin["amount"])     
        # Update sender user balance in the dataframe
        self.accounts.loc[self.accounts['user_address'] == senderAddr, coin["denom"]] = sender_balance - float(coin["amount"])     
        
        # LOgging
        new_sender_balance = float(self.accounts.loc[self.accounts['user_address'] == senderAddr][coin["denom"]])        
        new_recepient_balance = float(self.accounts.loc[self.accounts['user_address'] == recipientAddr][coin["denom"]])        
#         print(f"{float(coin['amount'])} {coin['denom']} TRANSFERRED FROM {senderAddr} TO {recipientAddr}")
        return None
    
    
    # TRANSFER TOKENS
    def SendCoinsFromAccountToModule(self, senderAddr, recipientAddr,  coin):
        sender = self.accounts.loc[self.accounts['user_address'] == senderAddr]
        if len(sender) == 0: # Return Error 
            return "Sender doesn't exist"

        # balance check
        sender_balance = float(self.accounts.loc[self.accounts['user_address'] == senderAddr][coin["denom"]])        
        if sender_balance < float(coin["amount"]):
            return "insufficient balance"        
        
        # Get recepient
        recepient = self.accounts.loc[self.accounts['user_address'] == recipientAddr]
        if len(recepient) == 0: # Create user 
            self.accounts.loc[len(self.accounts.index)] = [recipientAddr,0,0,0,0]
        recepient = self.accounts.loc[self.accounts['user_address'] == recipientAddr]
        
        # Update recepient user balance in the dataframe
        recepient_balance = float(recepient[coin['denom']])
        self.accounts.loc[self.accounts['user_address'] == recipientAddr, coin["denom"]] = recepient_balance + float(coin["amount"])     
        # Update sender user balance in the dataframe
        self.accounts.loc[self.accounts['user_address'] == senderAddr, coin["denom"]] = sender_balance - float(coin["amount"])     
        
        # LOgging
        new_sender_balance = float(self.accounts.loc[self.accounts['user_address'] == senderAddr][coin["denom"]])        
        new_recepient_balance = float(self.accounts.loc[self.accounts['user_address'] == recipientAddr][coin["denom"]])        
#         print(f"{int(coin['amount'])} {coin['denom']} TRANSFERRED FROM {senderAddr} TO {recipientAddr}")
        return None
    
    
    # TRANSFER TOKENS
    def SendCoinsFromModuleToAccount(self, senderAddr, recipientAddr,  coin):
        sender = self.accounts.loc[self.accounts['user_address'] == senderAddr]
#         print(sender)
        if len(sender) == 0: # Return Error 
            return "Sender doesn't exist"

        # balance check
        sender_balance = float(self.accounts.loc[self.accounts['user_address'] == senderAddr][coin["denom"]])        
        if sender_balance < float(coin["amount"]):
            return "insufficient balance"        
        
        # Get recepient
        recepient = self.accounts.loc[self.accounts['user_address'] == recipientAddr]
#         print(recepient)
        if len(recepient) == 0: # Create user 
            self.accounts.loc[len(self.accounts.index)] = [recipientAddr,0,0,0,0]
        recepient = self.accounts.loc[self.accounts['user_address'] == recipientAddr]
        
        # Update recepient user balance in the dataframe
        recepient_balance = float(recepient[coin['denom']])
        self.accounts.loc[self.accounts['user_address'] == recipientAddr, coin["denom"]] = recepient_balance + float(coin["amount"])     
        # Update sender user balance in the dataframe
        self.accounts.loc[self.accounts['user_address'] == senderAddr, coin["denom"]] = sender_balance - float(coin["amount"])     
        
        # LOgging
        new_sender_balance = float(self.accounts.loc[self.accounts['user_address'] == senderAddr][coin["denom"]])        
        new_recepient_balance = float(self.accounts.loc[self.accounts['user_address'] == recipientAddr][coin["denom"]])        
#         print(f"{int(coin['amount'])} {coin['denom']} TRANSFERRED FROM {senderAddr} TO {recipientAddr}")
        return None    
    
    
    # MINT NEW TOKENS
    def MintCoins(self, recipientAddr, coin):
        # get recepient
        recepient = self.accounts.loc[self.accounts['user_address'] == recipientAddr]
        if len(recepient) == 0: # Create user 
            self.accounts.loc[len(self.accounts.index)] = [recipientAddr,0,0,0,0]
        recepient = self.accounts.loc[self.accounts['user_address'] == recipientAddr]

        # Update recepient user balance in the dataframe
        cur_balance = float(recepient[coin['denom']])
        self.accounts.loc[self.accounts['user_address'] == recipientAddr, coin["denom"]] = cur_balance + float(coin["amount"])     
        
        # Update total accounted token supply
        self.totalSupply[coin["denom"]] = self.totalSupply[coin["denom"]] + coin["amount"]

        # LOgging
        new_balance = float(self.accounts.loc[self.accounts['user_address'] == recipientAddr][coin["denom"]])
#         print(f"{int(coin['amount'])} {coin['denom']} MINTED")
        return None

       
    # BURN TOKENS
    def BurnCoins(self, senderAddr, coin):
        # get sender
        sender = self.accounts.loc[self.accounts['user_address'] == senderAddr]
        if len(sender) == 0: # Return Error 
            return "Sender doesn't exist"
        # balance check
        sender_balance = float(self.accounts.loc[self.accounts['user_address'] == senderAddr][coin["denom"]])        
        if sender_balance < float(coin["amount"]):
            return "insufficient balance"                
        
        # Update sender user balance in the dataframe
        cur_balance = float(sender[coin['denom']])
        self.accounts.loc[self.accounts['user_address'] == senderAddr, coin["denom"]] = cur_balance - float(coin["amount"])     
        
        # Update total accounted token supply
        self.totalSupply[coin["denom"]] = self.totalSupply[coin["denom"]] - coin["amount"]

        # LOgging
        new_balance = float(self.accounts.loc[self.accounts['user_address'] == senderAddr][coin["denom"]])
#         print(f"{float(coin['amount'])} {coin['denom']} BURNT")
        return None

           
        
    # GET USER BALANCE
    def GetBalance(self, addr, denom):
        user = self.accounts.loc[self.accounts['user_address'] == addr]
        len_ = len(user)
        if len(user) == 0:
            self.accounts.loc[len(self.accounts.index)] = [addr,0,0,0,0]
            return 0
        return float(user[denom])
        

    def GetTotalSupply(self, denom):
        return self.totalSupply[denom]
    
    
    def SpendableCoins(self,ctx, addr):
        pass

    def IsSendEnabledCoin(self,ctx, coin):
        pass

### Terra Classic implemented as a class for simulation purposes

- **ASTROPORT ::: LUNA <> TERRA POOL** :  Modified astroport's xyk pool which checks if an arbitrage is possible between pool <> Market module, and if so executes the arb and the profit amount is indexed for analysis purposes. 
    - The astroport's pool functions are implemented directly as part of the terra classic class here. 
    - Code inspired from https://github.com/astroport-fi/astroport-core/tree/main/contracts
    
<br>    

- **TWAP Oracle Contract** : A TWAP feed for LUNA <> UST exchange rate via the astroport pool has been added to the market module's logic. TWAP functions are directly implemented as part of the terra classic class here.
    - Code inspired from https://github.com/mars-protocol/mars-core/blob/master/contracts/mars-oracle/src/contract.rs

<br>

- **Market Module** : Terra's Market module logic. Implemented directly as functions here. 

- **Oracle and Bank Modules** : These modules are defined above and are part of the terra classic implementation here as a defined instance 



In [468]:
#---------------x-----------x--------------
class Simulated_Terra_Classic_Instance:
    
    def __new__(cls, *args, **kwargs):
        return super().__new__(cls)  
    
    #---------------------------------------------------
    #------------ Intitialization Functions ------------
    #---------------------------------------------------
    
    # Intitialize ASTROPORT's UST-LUNA pool
    def init_astroport_pool(self, uluna_pool_size, ust_pool_size, lp_supply, block_time_last, uluna_cumulative_last, ust_cumulative_last):
        self.DEFAULT_SLIPPAGE = 0.005
        self.MAX_ALLOWED_SLIPPAGE = 0.5
        self.TOTAL_FEE = 0.03
        self.MAKER_COMMISSION_PERCENT = 0
        
        self.uluna_pool_size =  uluna_pool_size
        self.ust_pool_size  = ust_pool_size
        self.lp_supply = lp_supply
        self.block_time_last = block_time_last
        self.uluna_cumulative_last = uluna_cumulative_last
        self.ust_cumulative_last = ust_cumulative_last    
    
    # Intitialize TWAP Oracle contract
    def init_twap_oracle(self, window_size, tolerance):
        self.window_size = window_size
        self.tolerance = tolerance
        self.snapshots = {
            "ust": [],
            "uluna": []
        }
            
    # Initialize Terra Classic's Market Module 
    def init_market_module(self, basepool, minstabilityspread, poolrecoveryperiod):

        self.basepool = basepool
        self.minstabilityspread = minstabilityspread
        self.poolrecoveryperiod = poolrecoveryperiod            
        self.delta = 0
    
        self.MicroLunaDenom = "uluna" 
        self.MicroUSDDenom  = "ust"
        self.MicroSDRDenom  = "usdr"
    
        # Keeps track of token balances
        self.BankKeeper = TerraBankModule()
        # Keeps track of oracle prices
        self.OracleKeeper = TerraOracleKeeper()
   
    #--------------------------------------------------
    #------------ Astroport Pool Functions ------------
    #--------------------------------------------------    
    
    # PROVIDE LIQUIDITY TO THE POOL
    def astroport_provide_liquidity(self, trader, ust_amount, uluna_amount, cur_block_time, slippage_tolerance):
        lp_to_mint = 0
        
        if self.lp_supply == 0:
            lp_to_mint = math.sqrt(ust_amount * uluna_amount)
        
        else:                                            
            #  Assert slippage tolerance
            slip_er = self._assert_slippage_tolerance(slippage_tolerance, ust_amount, uluna_amount)
            if slip_er != None:
                return { "amount":0, "error": slip_er }
            
            # Calculate LP tokens to mint
            ust_calc_lp_share = math.sqrt( ust_amount * (self.lp_supply/self.ust_pool_size) )
            uluna_calc_lp_share = math.sqrt( uluna_amount * (self.lp_supply/self.uluna_pool_size) )            
            lp_to_mint = min(uluna_calc_lp_share, ust_calc_lp_share)
            
        # Accumulate prices 
        accumulate_prices_response = self._accumulate_prices(cur_block_time)
        print(accumulate_prices_response)
        if accumulate_prices_response["needed"] != None:
            self.uluna_cumulative_last = accumulate_prices_response["uluna_cumulative_last"]
#             print(self.uluna_cumulative_last)
            self.ust_cumulative_last = accumulate_prices_response["ust_cumulative_last"]
#             print(self.ust_cumulative_last)
            self.block_time_last = accumulate_prices_response["block_time"]
#             print(self.block_time_last)
        
        # Update pool balances & LP supply
        self.uluna_pool_size = self.uluna_pool_size +  uluna_amount
        self.ust_pool_size = self.ust_pool_size + ust_amount
        self.lp_supply = self.lp_supply + lp_to_mint
        
        # Process token transfers
        self.BankKeeper.SendCoinsFromModuleToModule(trader, "astroport_ust_luna_pool", {"denom":"uluna", "amount":uluna_amount })
        self.BankKeeper.SendCoinsFromModuleToModule(trader, "astroport_ust_luna_pool", {"denom":"ust", "amount":ust_amount })
        self.BankKeeper.MintCoins(trader,{"denom":"ust_uluna_lp_token", "amount":lp_to_mint })
        
        return { "lp_to_mint":lp_to_mint, "error": None }
       
        
    # REMOVE LIQUIDITY FROM THE POOL        
    def astroport_remove_liquidity(self, trader, lp_share_to_burn, cur_block_time):   
        # calculate tokens to be refunded
        refund_uluna = self.uluna_pool_size * (lp_share_to_burn/self.lp_supply)
        refund_ust = self.ust_pool_size * (lp_share_to_burn/self.lp_supply)

        # Accumulate prices 
        accumulate_prices_response = self._accumulate_prices(cur_block_time)
        if accumulate_prices_response["needed"] != None:
            self.uluna_cumulative_last = accumulate_prices_response["uluna_cumulative_last"]
            self.ust_cumulative_last = accumulate_prices_response["ust_cumulative_last"]
            self.block_time_last = accumulate_prices_response["block_time"]        
        
        # Update pool balances & LP supply
        self.uluna_pool_size = self.uluna_pool_size - refund_uluna
        self.ust_pool_size = self.ust_pool_size - refund_ust
        self.lp_supply = self.lp_supply - lp_share_to_burn
        
        # Process token transfers
        self.BankKeeper.SendCoinsFromModuleToModule("astroport_ust_luna_pool", trader , {"denom":"uluna", "amount":refund_uluna })
        self.BankKeeper.SendCoinsFromModuleToModule("astroport_ust_luna_pool", trader, {"denom":"ust", "amount":refund_ust })
        self.BankKeeper.BurnCoins(trader, {"denom":"ust_uluna_lp_token", "amount":lp_share_to_burn })
        
        return { "refund_uluna":refund_uluna, "refund_ust": refund_ust, "error": None }        

    
    # Execute Swap via Astroport's Pool
    def astroport_swap(self, trader, offer_asset, cur_timestamp, belief_price, max_spread):
        offer_pool = 0
        ask_pool = 0
        
        offer_asset = offer_asset
        ask_asset = {"denom": "", "amount":0}
        print(ask_asset)
        
        # get offer and ask pools
        if offer_asset["denom"] == "ust":
            offer_pool = self.ust_pool_size
            ask_pool = self.uluna_pool_size
            ask_asset["denom"] = "uluna"
        elif offer_asset["denom"] == "uluna":
            offer_pool = self.uluna_pool_size
            ask_pool = self.ust_pool_size
            ask_asset["denom"] = "ust"
        else:
            return { "return_asset": ask_asset, "expected_profit_asset": ask_asset,"error": f"unsupported offer denom: {offer_asset['denom']}" }
        print(ask_asset)
        
        # calculate swap amount
        swap_response = self._astroport_compute_swap(offer_pool, ask_pool, offer_asset["amount"])
        print("swap_response")
        print(swap_response)
        
        # check spread assertion
        spread_error = self._astroport_assert_max_spread(belief_price, max_spread, offer_asset["amount"], swap_response["return_amount"], swap_response["spread_amount"])
        if spread_error["error"] != None:
            return { "return_asset": ask_asset, "expected_profit_asset": ask_asset, "error": spread_error }
            
        # calculate tax
        tax_response = self._astroport_compute_tax(ask_asset["denom"], swap_response["return_amount"])
        tax_amount = tax_response["tax"]
        print(f"tax_amount = {tax_amount}")
        
        # calculate Astroport Maker Fee 
        maker_fee_in_return_asset = swap_response["commission_amount"] * self.MAKER_COMMISSION_PERCENT
        print(f"maker_fee_in_return_asset = {maker_fee_in_return_asset}")
        
        # Check if any arbitrage is possible between the Pool and Market Module's Virtual pool
        # --> Use offer amount for arbitrage, basically check if the amount returned by the 
        # market module swap if greater than what the astroport pool is offering to the user, and if
        # so then arb exists and execute it. Difference between the amount returned by the module and the 
        # amount which will be transferred to maker & user is the profit. 
        #-----------------------------
        # --> Example : Assume 1 LUNA = $93 USD via Oracle and 1 LUNA = 150 UST on Astroport (implying UST = 93/150=$0.62)
        # User will be selling LUNA on Astroport to get cheap UST and then burn it at a higher price to increase their LUNA holdings
        # if user swaps 100 LUNA for 150,00 UST, the swap internally checks that 150,00 UST will get it 161.2 LUNA, so a profit of 61.2 LUNA, 
        # so the pool will execute the arb and transfer the profits as per some defined logic. 
        #-----------------------------
        # --> Example : Assume 1 LUNA = $93 USD via Oracle and 1 LUNA = 80 UST on Astroport (implying UST = 93/150=$1.16)
        # User will be selling UST on Astroport to get cheap LUNA and then burn it at a higher price to increase their UST holdings
        # if user swaps 1000 UST for 12.5 LUNA, the swap internally checks that 12.5 LUNA will get it 1162.5 UST, so a profit of 162.5 UST, 
        # so the pool will execute the arb and transfer the profits as per some defined logic. 
        is_arb_possible = False
        expected_profit_asset = {"denom": "ust", "amount":0}
        offer_asset_arb = None            
        #------- Offer the ask asset to module and check if arb is possible or not        
        arb_offer_asset = {"denom": ask_asset["denom"], "amount": swap_response["return_amount"] + swap_response["commission_amount"]  }
        print(f"arb_offer_asset = {arb_offer_asset}")
        arb_ask_denom = offer_asset["denom"]
        simulate_arb_swap = self._simulate_arb_with_market_module(arb_offer_asset, arb_ask_denom, cur_timestamp )
        print(f"simulate_arb_swap = {simulate_arb_swap}")
        if simulate_arb_swap["return_amount"] > offer_asset["amount"]:
            is_arb_possible = True
            expected_profit_asset = {"denom": offer_asset["denom"] , "amount":(simulate_arb_swap["return_amount"] - offer_asset["amount"])  }
        #------- Offer the offer asset to module and check if arb is possible or not        
        else:
            arb_offer_asset = offer_asset
            arb_ask_denom = ask_asset["denom"]
            print(f"arb_offer_asset = {arb_offer_asset}")
            simulate_arb_swap = self._simulate_arb_with_market_module(arb_offer_asset, arb_ask_denom, cur_timestamp)
            print(f"simulate_arb_swap = {simulate_arb_swap}")
            if simulate_arb_swap["return_amount"] >  (swap_response["return_amount"] + swap_response["commission_amount"]) :
                is_arb_possible = True
                profit_amount = simulate_arb_swap["return_amount"] - (swap_response["return_amount"] + swap_response["commission_amount"])
                expected_profit_asset = {"denom": ask_asset["denom"] , "amount":profit_amount }
            print(f"expected_profit_asset = {expected_profit_asset}") 

        # Execute Arbitrage with Market Module
        arb_response = None
        if is_arb_possible:
            arb_response = self.execute_arb_with_market_module(arb_offer_asset, arb_ask_denom, cur_timestamp)
            print(f"arb_response = {arb_response}") 
    
        # Accumulate prices 
        accumulate_prices_response = self._accumulate_prices(cur_block_time)
        if accumulate_prices_response["needed"] != None:
            self.uluna_cumulative_last = accumulate_prices_response["uluna_cumulative_last"]
            self.ust_cumulative_last = accumulate_prices_response["ust_cumulative_last"]
            self.block_time_last = accumulate_prices_response["block_time"]        
        
        # Update pool balances
        if offer_asset["denom"] == "ust":
            self.ust_pool_size = self.ust_pool_size + offer_asset["amount"]
            self.uluna_pool_size = self.uluna_pool_size - swap_response["return_amount"] - maker_fee_in_return_asset
        else:
            self.uluna_pool_size = self.uluna_pool_size + offer_asset["amount"]
            self.ust_pool_size = self.ust_pool_size - swap_response["return_amount"] - maker_fee_in_return_asset
            
        ask_asset["amount"] = swap_response["return_amount"]
        print(f"ask_asset = {ask_asset}") 
        
        # Process token transfers
        self.BankKeeper.SendCoinsFromModuleToModule(trader, "astroport_ust_luna_pool", offer_asset)
        self.BankKeeper.SendCoinsFromModuleToModule("astroport_ust_luna_pool", trader, ask_asset)
        self.BankKeeper.SendCoinsFromModuleToModule("astroport_ust_luna_pool", "astroport_pool_arb_profits", expected_profit_asset )
                
        return { "return_asset": ask_asset,  "expected_profit_asset":expected_profit_asset, "error": spread_error }
            
                    
            
    # Compute ask amount given the offer amount for swap via the Astroport pool
    def _astroport_compute_swap(self, offer_pool, ask_pool, offer_amount):
        # ask_amount = (ask_pool - cp / (offer_pool + offer_amount))
        cp = ask_pool * offer_pool
        return_amount = ask_pool - cp/(offer_pool + offer_amount)
        
        # Calculate spread & commission
        spread_amount = offer_amount * (ask_pool/offer_pool) - return_amount
        commission_amount = return_amount * self.TOTAL_FEE
        
        return_amount = return_amount - commission_amount
        return {
            "return_amount": return_amount,
            "spread_amount": spread_amount,
            "commission_amount": commission_amount            
        }
    
    # Compute offer amount given the ask amount for swap via the Astroport pool
    def _astroport_compute_offer_amount(self, offer_pool, ask_pool, ask_amount):
        # offer_amount = cp / (ask_pool - ask_amount / (1 - commission_rate)) - offer_pool
        cp = ask_pool * offer_pool
        one_minus_commission = 1 - self.TOTAL_FEE
        inv_one_minus_commission = 1/one_minus_commission        
        offer_amount = (cp / (ask_pool - (ask_amount*inv_one_minus_commission))) - offer_pool
        
        return offer_amount
    
    # Assert spread check
    def _astroport_assert_max_spread(self, belief_price, max_spread, offer_amount, return_amount, spread_amount):
        default_spread = self.DEFAULT_SLIPPAGE
        max_allowed_spread = self.MAX_ALLOWED_SLIPPAGE
        
        # basic checks
        if not max_spread or max_spread == 0:
            max_spread = default_spread
        elif max_spread > max_allowed_spread:
            return {
                "error": "_astroport_assert_max_spread"
            }
        
        # if belief price is provided
        if belief_price > 0:
            expected_return = offer_amount * (1/belief_price)
            spread_amount = expected_return - return_amount
            if (return_amount < expected_return) and ((spread_amount/expected_return) > max_spread):
                return { "error": "spread check failed" }
        # if price is not provided
        elif (spread_amount/(return_amount + spread_amount)) > max_spread :
            spr = (spread_amount/(return_amount + spread_amount))
            print(f"spread_amount = {spread_amount} | return_amount = {return_amount} | spread_amount = {spread_amount}")
            print(f"max_spread = {max_spread} | spread = {spr}")
            return {  "error": "spread check failed" }
        
        return {  "error": None }
    
    def _assert_slippage_tolerance(self, _slippage_tolerance, ust_amount, uluna_amount):
        slippage_tolerance = self.DEFAULT_SLIPPAGE
        if _slippage_tolerance and _slippage_tolerance > 0:
            slippage_tolerance = _slippage_tolerance
        
        #  Ensure each price does not change more than what the slippage tolerance allows
        one_minus_slippage_tolerance = 1 - slippage_tolerance  
        if (((ust_amount/uluna_amount) * one_minus_slippage_tolerance) > (self.ust_pool_size/self.uluna_pool_size))\
        or (((uluna_amount/ust_amount) * one_minus_slippage_tolerance) > (self.uluna_pool_size/self.ust_pool_size)):
            return "_assert_slippage_tolerance :: Error"        
        return None

    def _astroport_compute_tax(self, denom, amount):
        return {"tax": 0}
    
    # Accumulate prices for the pool
    def _accumulate_prices(self, cur_block_time):
        if self.block_time_last >= cur_block_time:
            return {"uluna_cumulative_last":0, "ust_cumulative_last":0, "block_time":0, "needed": None}
        
        # time elapsed
        time_elapsed = cur_block_time - self.block_time_last
        
        pcl0 = self.uluna_cumulative_last
        pcl1 = self.ust_cumulative_last
                
        # accumulate prices
        if self.ust_pool_size > 0 and self.uluna_pool_size > 0:
            pcl0 =  self.uluna_cumulative_last + (time_elapsed * (self.ust_pool_size/self.uluna_pool_size))
            pcl1 =  self.ust_cumulative_last + (time_elapsed * (self.uluna_pool_size/self.ust_pool_size))            
        return {"uluna_cumulative_last":pcl0, "ust_cumulative_last":pcl1, "block_time":cur_block_time, "needed": True}

    # SIMULATES SWAP via Market Module
    def _simulate_arb_with_market_module(self, offer_asset, ask_denom, cur_timestamp):
        simulation_response = self.simulateMarketModuleSwap(offer_asset, ask_denom, cur_timestamp)
        return_amount  = simulation_response["return_denom"]["amount"]
        spread = simulation_response["spread"]
        error = simulation_response["error"]
        print(simulation_response)
        return { "return_amount":return_amount, "spread":spread }
    
    # Executes Market SWAP via Market Module
    def execute_arb_with_market_module(self, offer_asset, askDenom, cur_timestamp):
        arb_response = self.ExecuteMarketModuleSwap("astroport_ust_luna_pool",offer_asset, askDenom, cur_timestamp )
        return arb_response["SwapCoin"]
    
    def astroport_get_pool_info(self):
        return {
            "uluna_pool_size": self.uluna_pool_size,
            "ust_pool_size": self.ust_pool_size,
            "lp_supply": self.lp_supply,
            "block_time_last": self.block_time_last,
            "uluna_cumulative_last": self.uluna_cumulative_last,
            "ust_cumulative_last": self.ust_cumulative_last
        }
    
    def astroport_get_cumulative_price(self, denom):
        if denom == "ust":
            return self.ust_cumulative_last
        else:
            return self.uluna_cumulative_last
    
    def astroport_get_cumulative_prices(self):
        return {
            "block_time_last": self.block_time_last,
            "uluna_cumulative_last": self.uluna_cumulative_last,
            "ust_cumulative_last": self.ust_cumulative_last
        }
    
    def astroport_simulate_swap(self, offer_asset):
        offer_pool = 0
        ask_pool = 0
        
        offer_asset = offer_asset
        ask_asset = {"denom": "", "amount":0}
        
        # get offer and ask pools
        if offer_asset["denom"] == "ust":
            offer_pool = self.ust_pool_size
            ask_pool = self.uluna_pool_size
            ask_asset["denom"] = "uluna"
        elif offer_asset["denom"] == "uluna":
            offer_pool = self.uluna_pool_size
            ask_pool = self.ust_pool_size
            ask_asset["denom"] = "ust"
        else:
            return 0
            
        # calculate swap amount
        swap_response = self._astroport_compute_swap(offer_pool, ask_pool, offer_asset["amount"])
        ask_asset["amount"] = swap_response["return_amount"]
        return ask_asset

    
    def astroport_simulate_reverse_swap(self, ask_asset):
        offer_pool = 0
        ask_pool = 0
        
        offer_asset = {"denom": "", "amount":0}
        ask_asset = ask_asset
        
        # get offer and ask pools
        if offer_asset["denom"] == "ust":
            offer_pool = self.ust_pool_size
            ask_pool = self.uluna_pool_size
            offer_asset["denom"] = "ust"
        elif offer_asset["denom"] == "uluna":
            offer_pool = self.uluna_pool_size
            ask_pool = self.ust_pool_size
            offer_asset["denom"] = "uluna"
        else:
            return 0
        
        offer_amount = self._astroport_compute_offer_amount(offer_pool, ask_pool, ask_asset["amount"])
        return offer_amount
        
            
    #--------------------------------------------------
    #------------ Oracle Contract Functions ------------
    #--------------------------------------------------    
    
        
    # RECORD TWAP SNAPSHOT
    def record_twap_snapshots(self, cur_block_time):
        # LOOP ACROSS LUNA AND UST SNAPSHOTS
        for denom, twap_snapshots in self.snapshots.items():
            print(f"\n{denom}")
            print(f"{twap_snapshots}")
            
            if len(twap_snapshots) > 0:
                latest_snapshot = twap_snapshots[len(twap_snapshots)-1]
                print(f"latest_snapshot = {latest_snapshot}")
                # A potential attack is to repeatly call `RecordTwapSnapshots` so that `snapshots` becomes a
                # very big vector, so that calculating the average price becomes extremely gas expensive.
                # To deter this, we reject a new snapshot if the most recent snapshot is less than `tolerance`
                # seconds ago.
                print(f"latest_snapshot = {cur_block_time - latest_snapshot['timestamp']} | tolerance = {self.tolerance}")
                if cur_block_time - latest_snapshot["timestamp"] < self.tolerance:
                    continue
        
            # Get cumulative price 
            print(f"Get cumulative price : {denom}")
            cumulative_price = self.astroport_get_cumulative_price(denom)
            print(f"cumulative_price = {cumulative_price} ")
        
            # Purge snapshots that are too old, i.e. more than (window_size + tolerance) away from the
            # current timestamp. These snapshots will never be used in the future for calculating
            # average prices        
            new_snapshots_array = []
            for snapshot in twap_snapshots:
                if (cur_block_time - snapshot["timestamp"]) <= self.window_size + self.tolerance:
                    print(f"snapshot still in array = {snapshot} ")
                    new_snapshots_array.append(snapshot)

            # Add latest snapshot
            new_snapshots_array.append({"timestamp":cur_block_time, "cumulative_price":cumulative_price})
            print(f"latest snapshot = {cumulative_price} ")
            print(new_snapshots_array)
            self.snapshots[denom] = new_snapshots_array
        
        return {"success":True}
    
    
    # QUERY TWAP PRICE
    def query_twap_price(self, denom, cur_block_time):
        # Get latest snapshot
        cumulative_price = self.astroport_get_cumulative_price(denom)
        print(cumulative_price)
        current_snapshot = {
            "timestamp": cur_block_time,
            "cumulative_price": cumulative_price
        }
        print(f"current_snapshot {current_snapshot}")
        
        # Find the oldest snapshot whose period from current snapshot is within the tolerable window
        # We do this using a linear search, and quit as soon as we find one; otherwise throw error
        previous_snapshot = {"timestamp":0, "cumulative_price": 0 }
        for snapshot in self.snapshots[denom]:
            print(snapshot)
            if self._period_diff(cur_block_time , snapshot["timestamp"], self.window_size) <= self.tolerance:
                previous_snapshot = snapshot

        if previous_snapshot["timestamp"] == 0:
            return {"twap":0, "error":"NoSnapshotWithinTolerance"}
                    
        price_delta = current_snapshot["cumulative_price"] - previous_snapshot["cumulative_price"]
        period = current_snapshot["timestamp"] - previous_snapshot["timestamp"]
        price = price_delta / period        
        return {"twap":price, "error":None}
                
    
    def _period_diff(self,a,b,c):
        return self._diff(  self._diff(a,b), c  )  
    
        
    def _diff(self, a,b):
        if a > b:
            return a - b
        else:
            return b - a

        
    #--------------------------------------------------
    #------------ Market Module Functions -------------
    #--------------------------------------------------    
            
      
     # Terra pool value is returned with following function:
    def get_TerraPool(self):
        terraPool = self.basepool + self.delta
        return terraPool
        
        
    # Luna pool can be retrived from Terra pool delta with following function:
    def get_LunaPool_fromTerraPool(self):
        terraPool = self.basepool + self.delta
        lunaPool = (self.basepool * self.basepool) / terraPool
        return lunaPool

    # GetBasePool returns the basepool
    def GetBasePool(self):
        return self.basepool
    
    # GetMinStabilitySpread returns the minstabilityspread
    def GetMinStabilitySpread(self):
        return self.minstabilityspread

    # GetPoolRecoveryPeriod returns the poolrecoveryperiod
    def GetPoolRecoveryPeriod(self):
        return self.poolrecoveryperiod
    
    # GetTerraPoolDelta returns the gap between the TerraPool and the TerraBasePool
    def GetTerraPoolDelta(self):
        return self.delta

    # SetTerraPoolDelta updates TerraPoolDelta which is gap between the TerraPool and the BasePool
    def _SetTerraPoolDelta(self,delta):
        self.delta = delta

    # USER INTERACTION - SWAP FUNCTION
    def ExecuteMarketModuleSwap(self, trader,offerCoin, askDenom, cur_timestamp ) : 
        res = self._handleSwapRequest(trader,offerCoin, askDenom, cur_timestamp )
        return res        

    # USER INTERACTION - SWAPSEND FUNCTION
    def ExecuteMarketModuleSwapSend(self, trader, offerCoin, askDenom, cur_timestamp ): 
        res = self._handleSwapRequest(trader,offerCoin, askDenom, cur_timestamp )
        return res
        
    # INTERNAL FUNCTION
    # // handleMsgSwap handles the logic of a MsgSwap
    # // This function does not repeat checks that have already been performed
    # // Ex) assert(offerCoin.Denom != askDenom)
    def _handleSwapRequest(self, trader,offerCoin, askDenom, cur_timestamp ): 

        #  Compute exchange rates between the ask and offer
        (swapDecCoin, spread, err) = self._ComputeMarketModuleSwap(offerCoin, askDenom, cur_timestamp)
        if err != None:
            return (None, err)

        #  Charge a spread if applicable; the spread is burned
        feeDecCoin : Coin
        if spread > 0:
            feeDecCoin = {"denom" : swapDecCoin["denom"], "amount" : spread * swapDecCoin["amount"]}
        else:
            feeDecCoin = {"denom" : swapDecCoin["denom"], amount : 0 }
        
        # Subtract fee from the swap coin
        swapDecCoin["amount"] = swapDecCoin["amount"] - feeDecCoin["amount"]

        # Update pool delta
        err = self._ApplySwapToPool(offerCoin, swapDecCoin, cur_timestamp)
        if err != None:
            return (None, err)

        # Send offer coins to module account
        err = self.BankKeeper.SendCoinsFromAccountToModule(trader, "market", offerCoin)
        if err != None:
            return None, err

        # Burn offered coins and subtract from the trader's account
        err = self.BankKeeper.BurnCoins("market", offerCoin)
        if err != None:
            return None, err

        # Mint asked coins and credit Trader's account
        #         swapCoin, decimalCoin = swapDecCoin.TruncateDecimal()
        #         feeDecCoin = feeDecCoin.Add(decimalCoin) # add truncated decimalCoin to swapFee
        #         feeCoin, _ := feeDecCoin.TruncateDecimal()
        mintCoins = { "denom": swapDecCoin["denom"], "amount":swapDecCoin["amount"] + feeDecCoin["amount"] }
        err = self.BankKeeper.MintCoins("market", mintCoins)
        if err != None:
            return None, err

        # Send swap coin to the trader
        err = self.BankKeeper.SendCoinsFromModuleToAccount("market", trader, swapDecCoin)
        if err != None:
            return None, err

        # Send swap fee to oracle account
        if feeDecCoin["amount"] > 0:
            err = self.BankKeeper.SendCoinsFromModuleToModule("market", "oracle", feeDecCoin)
            if err != None:
                return None, err


        return { "SwapCoin": swapDecCoin,
                  "SwapFee":  feeDecCoin,
                    "spread": spread,
                    "Error": None
                }


    
    # INTERNAL FUNCTION
    # // _ComputeMarketModuleSwap returns the amount of asked coins should be returned for a given offerCoin at the effective
    # // exchange rate registered with the oracle.
    # // Returns an Error if the swap is recursive, or the coins to be traded are unknown by the oracle, or the amount
    # // to trade is too small.
    def _ComputeMarketModuleSwap(self, offerCoin, askDenom, cur_timestamp):

        # Return invalid recursive swap err
        if offerCoin["denom"] == askDenom:
            return {"denom":"", "amount":0}, 0, "_ComputeMarketModuleSwap :: offer asset cannot be same as ask asset"

        # Swap offer coin to base denom (usdr) for simplicity of swap process
        baseOfferDecCoin, err = self._ComputeInternalSwap(offerCoin, self.MicroSDRDenom, cur_timestamp )
        if err != None:
            return {"denom":"", "amount":0}, 0, err
#         else:
#             print(f"{offerCoin['amount']} {offerCoin['denom']} --> {baseOfferDecCoin['amount']} {baseOfferDecCoin['denom']}")

        # Get Ask asset swap amount based on the oracle price
        retDecCoin, err = self._ComputeInternalSwap(baseOfferDecCoin, askDenom, cur_timestamp)
        if err != None:
            return {"denom":"", "amount":0}, 0, err
#         else:
#             print(f"{baseOfferDecCoin['amount']} {baseOfferDecCoin['denom']} --> {retDecCoin['amount']} {retDecCoin['denom']}")

        # Terra => Terra swap
        # Apply only tobin tax without constant product spread
        if offerCoin["denom"] != self.MicroLunaDenom and askDenom != self.MicroLunaDenom:
            # OfferCoin Toblin Tax
            offerTobinTax, err2 = self.OracleKeeper.GetTobinTax(offerCoin["denom"])
            if err2 != None :
                return {"denom":"", "amount":0}, 0, err2

            # AskAsset Toblin Tax
            askTobinTax, err2 = self.OracleKeeper.GetTobinTax(ctx, askDenom)
            if err2 != None :
                return {"denom":"", "amount":0}, 0, err2

            # Apply highest tobin tax for the denoms in the swap operation
            tobinTax = 0
            if askTobinTax > offerTobinTax:
                tobinTax = askTobinTax
            else :
                tobinTax = offerTobinTax
            
            # Return the computed returnAsset and spread for Terra --> Terra Swap
            spread = tobinTax
            return (retDecCoin, spread, None)

        basePool = self.basepool
        minSpread = self.minstabilityspread

        #  constant-product, which by construction is square of base(equilibrium) pool
        # Calculate current TerraPool and LunaPool values
        cp = basePool*basePool
        terraPoolDelta = self.GetTerraPoolDelta()
        terraPool = basePool + terraPoolDelta
        lunaPool = cp / terraPool
    
#         print(f"terraPool = {terraPool} lunaPool = {lunaPool} terraPoolDelta = {terraPoolDelta}")
    
        # Assign TerraPool / LunaPool to OfferAsset / AskAssets
        offerPool = "" # base denom(usdr) unit
        askPool = ""   # base denom(usdr) unit
        #  Terra->Luna swap
        if offerCoin["denom"] != self.MicroLunaDenom:
            offerPool = terraPool
            askPool = lunaPool
        # Luna->Terra swap
        else:
            offerPool = lunaPool
            askPool = terraPool

        # Get cp(constant-product) based swap amount
        # baseAskAmt = askPool - cp / (offerPool + offerBaseAmount)
        # baseAskAmt is base denom(usdr) unit
        exp_new_ask_pool = cp/(offerPool + baseOfferDecCoin["amount"])
        baseAskAmt = askPool - exp_new_ask_pool
#         print(f"askPool = {askPool} | exp_new_ask_pool = {exp_new_ask_pool} | baseAskAmt = {baseAskAmt} ")

        # Both baseOffer and baseAsk are usdr units, so spread can be calculated by
        # spread = (baseOfferAmt - baseAskAmt) / baseOfferAmt
        baseOfferAmount = baseOfferDecCoin["amount"]
        spread = (baseOfferAmount - baseAskAmt) / baseOfferAmount

        if spread < minSpread:
            spread = minSpread
#         print(f"spread = {spread} ")

        return(retDecCoin, spread, None)

    
    # INTERNAL FUNCTION
    # ComputeInternalSwap returns the amount of asked DecCoin should be returned for a given offerCoin at the effective
    # exchange rate registered with the oracle.
    # Different from ComputeSwap, ComputeInternalSwap does not charge a spread as its use is system internal.
    def _ComputeInternalSwap(self, offerCoin, askDenom, cur_timestamp):
        if offerCoin["denom"] == askDenom:
            return offerCoin, 0
        
        # Get exchange rate :: OfferAsset --> Luna
        offerRate, err = self.OracleKeeper.GetLunaExchangeRate(offerCoin["denom"])
        # We use average of Oracle price and TWAP price for non-usdr coins
        if offerCoin["denom"] != "usdr":            
            twap_rate = self.query_twap_price(offerCoin["denom"], cur_timestamp)
            if twap_rate["error"] != None:
                return {"denom":"", "amount":0}, f"{offerCoin['denom']} - {twap_rate['error']}"
            twap_rate = twap_rate["twap"]
            offerRate = (offerRate + twap_rate)/2
            
        if err != None:
            return {"denom":"", "amount":0}, f"ErrNoEffectivePriceFromOracleFor ${offerCoin['denom']}"
        
        # Get exchange rate :: AskAsset --> Luna
        askRate, err = self.OracleKeeper.GetLunaExchangeRate(askDenom)
        # We use average of Oracle price and TWAP price for non-usdr coins
        if askDenom != "usdr":
            twap_rate = self.query_twap_price(askDenom, cur_timestamp)
            if twap_rate["error"] != None:
                return {"denom":"", "amount":0}, f"{askDenom} - {twap_rate['error']}"
            twap_rate = twap_rate["twap"]            
#             print(twap_rate)
            askRate = (askRate + twap_rate)/2

        if err != None:
            return {"denom":"", "amount":0}, f"ErrNoEffectivePriceFromOracleFor ${askDenom}"
        
        # Calculate return amount
        retAmount = offerCoin["amount"] * askRate / offerRate
        if retAmount < 0:
            return {"denom":"", "amount":0}, "ComputeInternalSwap::Err Return Calc"
        
        # return calc. return amount
        return ({"denom":askDenom,"amount": retAmount}, None)
    
    
    # INTERNAL FUNCTION
    # ApplySwapToPool updates each pool with offerCoin and askCoin taken from swap operation,
    # OfferPool = OfferPool + offerAmt (Fills the swap pool with offerAmt)
    # AskPool = AskPool - askAmt       (Uses askAmt from the swap pool)
    def  _ApplySwapToPool(self, offerCoin, askCoin, cur_timestamp):
        # No delta update in case Terra to Terra swap
        if offerCoin["denom"] != self.MicroLunaDenom and askCoin["denom"] != self.MicroLunaDenom:
            return None
        # Get Delta
        terraPoolDelta = self.GetTerraPoolDelta()

        # In case swapping Terra to Luna, the terra swap pool(offer) must be 
        # increased and the luna swap pool(ask) must be decreased
        if offerCoin["denom"] != self.MicroLunaDenom and askCoin["denom"] == self.MicroLunaDenom:
            offerBaseCoin, err = self._ComputeInternalSwap(offerCoin, self.MicroSDRDenom, cur_timestamp)
            if err != None:
                return err
            terraPoolDelta = terraPoolDelta + offerBaseCoin["amount"]

        # In case swapping Luna to Terra, the luna swap pool(offer) must be increased and the terra swap pool(ask) must be decreased
        if offerCoin["denom"] == self.MicroLunaDenom and askCoin["denom"] != self.MicroLunaDenom:
            askBaseCoin, err = self._ComputeInternalSwap(askCoin, self.MicroSDRDenom, cur_timestamp)
            if err != None:
                return err
            terraPoolDelta = terraPoolDelta - askBaseCoin["amount"]
        
        # Update Terra Delta Variable
        self._SetTerraPoolDelta(terraPoolDelta)

        return None
      
    # EXTERNAL QUERY FUNCTION : DOESN'T IMPACT STATE
    # simulateSwap interface for simulate swap
    def simulateMarketModuleSwap(self, offerCoin, askDenom, cur_timestamp):
        if askDenom == offerCoin["denom"] :
            return {"denom":"", "amount":0}, "askDenom and offerDenom cannot be same"
        
        # Calculate Swap
        (swapCoin, spread, err) = self._ComputeMarketModuleSwap(offerCoin, askDenom, cur_timestamp)
        if err != None:
            return { 
                "return_denom": {"denom":"", "amount":0}, 
                "spread": 0,
                "error": err
            }
        
        # Subtract spread
        if spread > 0:
            swapFeeAmt = spread * swapCoin["amount"]
            if swapFeeAmt > 0:
                swapFee = { "denom":swapCoin["denom"], "amount":swapFeeAmt }
                swapCoin["amount"] = swapCoin["amount"] -  swapFee["amount"]

        return {
            "return_denom": swapCoin,
            "spread": spread,
            "error": None
        }
    

    # END--BLOCK FUNCTION    
    # ==> EndBlocker is called at the end of every block
    def TerraMarketModuleEndBlock(self):
        # Replenishes each pools towards equilibrium
        self._ReplenishPools()
    
    # INTERNAL FUNCTION
    # Replenishes each pools towards equilibrium
    def _ReplenishPools(self):
        # Get current pool delta
        poolDelta = self.GetTerraPoolDelta()
        # Calculate Pool Regression Amount with current Pool Recovery period
        poolRegressionAmt = poolDelta / self.poolrecoveryperiod
        # Replenish terra pool towards base pool. 
        # regressionAmt cannot make delta zero
        newPoolDelta = poolDelta - poolRegressionAmt
        # Update Delta
        self._SetTerraPoolDelta(newPoolDelta)
#         print(f"Replenishing pools : poolDelta = {poolDelta} | poolRegressionAmt = {poolRegressionAmt} | newPoolDelta = {newPoolDelta}")

        
        


 
    
    
    
    
    

<!--  "window_size": 1920,
    "tolerance": 120 -->


In [469]:
# Market Module parameters
init_basepool = 50_000_000   # 50 M SDR
init_minstabilityspread = 0.05    # 5%
init_poolrecoveryperiod = 36      # regression parameter

# TWAP parameters
twap_window_size = 90 #1800 # 30 min
twap_tolerance = 12    # 120 sec

# Astroport Pool parameters
init_astroport_uluna_pool_size = 0
init_astroport_ust_pool_size = 0
init_lp_supply = 0
block_time_last = 0
uluna_cumulative_last = 0
ust_cumulative_last = 0


In [470]:
# CREATE TERRA CLASSIC INSTANCE 
terra_classic_instance = Simulated_Terra_Classic_Instance()

# Initialize Market Module, Oracle Module and Bank Modules
terra_classic_instance.init_market_module(init_basepool, init_minstabilityspread, init_poolrecoveryperiod)
terra_classic_instance.BankKeeper.MintCoins("trader",{"denom":"uluna", "amount":800 * 10**6 })
terra_classic_instance.BankKeeper.MintCoins("trader",{"denom":"ust", "amount":18000 * 10**6 })

# # Set initial exchange rates via Oracle Module
terra_classic_instance.OracleKeeper.SetLunaExchangeRate("ust", 3.074 )
terra_classic_instance.OracleKeeper.SetLunaExchangeRate("usdr", 2.290 )


# Initialize Market Module, Oracle Module and Bank Modules
terra_classic_instance.init_astroport_pool(init_astroport_uluna_pool_size, init_astroport_ust_pool_size, \
                                           init_lp_supply, block_time_last, uluna_cumulative_last, ust_cumulative_last)
terra_classic_instance.init_twap_oracle(twap_window_size, twap_tolerance)


In [471]:
# Provide initial Liquidity to the pool :: 800 M UST + 7 M LUNA
terra_classic_instance.astroport_provide_liquidity("trader", 80000, 700, cur_block_time, 0 )
terra_classic_instance.astroport_provide_liquidity("trader", 800_000000, 7_000000, cur_block_time, 0 )


{'uluna_cumulative_last': 0, 'ust_cumulative_last': 0, 'block_time': 1649112672, 'needed': True}
{'uluna_cumulative_last': 0, 'ust_cumulative_last': 0, 'block_time': 0, 'needed': None}


{'lp_to_mint': 8650.615454144221, 'error': None}

In [493]:
for i in range(0,100):
    cur_block_time = cur_block_time + 7
    terra_classic_instance.astroport_provide_liquidity("trader", 800, 7, cur_block_time, 0 )
    terra_classic_instance.record_twap_snapshots(cur_block_time)
    ust_twap_response = terra_classic_instance.query_twap_price("ust",cur_block_time)
    uluna_twap_response = terra_classic_instance.query_twap_price("uluna",cur_block_time)
    print(f"ust_twap_response = {ust_twap_response} | uluna_twap_response = {uluna_twap_response}")
    if ust_twap_response["twap"] > 0 

{'uluna_cumulative_last': 4800.0, 'ust_cumulative_last': 0.36750000000000005, 'block_time': 1649112714, 'needed': True}


{'lp_to_mint': 0.12701524060464991, 'error': None}


ust
[{'timestamp': 1649112679, 'cumulative_price': 0.061250000000000006}, {'timestamp': 1649112693, 'cumulative_price': 0.18375000000000002}, {'timestamp': 1649112707, 'cumulative_price': 0.30625}]
latest_snapshot = {'timestamp': 1649112707, 'cumulative_price': 0.30625}
latest_snapshot = 7 | tolerance = 12

uluna
[{'timestamp': 1649112679, 'cumulative_price': 800.0}, {'timestamp': 1649112693, 'cumulative_price': 2400.0}, {'timestamp': 1649112707, 'cumulative_price': 4000.0}]
latest_snapshot = {'timestamp': 1649112707, 'cumulative_price': 4000.0}
latest_snapshot = 7 | tolerance = 12


{'success': True}

0.36750000000000005

0.36750000000000005
current_snapshot {'timestamp': 1649112714, 'cumulative_price': 0.36750000000000005}
{'timestamp': 1649112679, 'cumulative_price': 0.061250000000000006}
{'timestamp': 1649112693, 'cumulative_price': 0.18375000000000002}
{'timestamp': 1649112707, 'cumulative_price': 0.30625}


{'twap': 0, 'error': 'NoSnapshotWithinTolerance'}

4800.0
current_snapshot {'timestamp': 1649112714, 'cumulative_price': 4800.0}
{'timestamp': 1649112679, 'cumulative_price': 800.0}
{'timestamp': 1649112693, 'cumulative_price': 2400.0}
{'timestamp': 1649112707, 'cumulative_price': 4000.0}


{'twap': 0, 'error': 'NoSnapshotWithinTolerance'}

In [465]:
terra_classic_instance.astroport_simulate_swap({"denom":"uluna","amount":1})

{'denom': 'ust', 'amount': 110.85712699890136}

In [466]:
terra_classic_instance.astroport_swap("trader", {"denom":"uluna","amount":1},0, 0,0)



{'denom': '', 'amount': 0}
{'denom': 'ust', 'amount': 0}
swap_response
{'return_amount': 110.85712699890136, 'spread_amount': 1.6348702573054652e-05, 'commission_amount': 3.4285709381103513}
tax_amount = 0
maker_fee_in_return_asset = 0.0
arb_offer_asset = {'denom': 'ust', 'amount': 114.28569793701172}
1.0412500000000002
current_snapshot {'timestamp': 0, 'cumulative_price': 1.0412500000000002}
{'timestamp': 1649112574, 'cumulative_price': 0.18375000000000002}
{'timestamp': 1649112588, 'cumulative_price': 0.30625}
{'timestamp': 1649112602, 'cumulative_price': 0.4287500000000001}
{'timestamp': 1649112616, 'cumulative_price': 0.5512500000000001}
{'timestamp': 1649112630, 'cumulative_price': 0.6737500000000002}
{'timestamp': 1649112644, 'cumulative_price': 0.7962500000000002}
{'timestamp': 1649112658, 'cumulative_price': 0.9187500000000003}
{'timestamp': 1649112672, 'cumulative_price': 1.0412500000000002}
{'return_denom': {'denom': '', 'amount': 0}, 'spread': 0, 'error': 'ust - NoSnapshotWi

{'return_asset': {'denom': 'ust', 'amount': 110.85712699890136},
 'expected_profit_asset': {'denom': 'ust', 'amount': 0},
 'error': {'error': None}}

In [467]:
terra_classic_instance.ExecuteMarketModuleSwap("trader", {"denom":"uluna","amount":1}, "ust", cur_block_time)
# (self, trader,offerCoin, askDenom, cur_timestamp )

13600.0
current_snapshot {'timestamp': 1649112672, 'cumulative_price': 13600.0}
{'timestamp': 1649112574, 'cumulative_price': 2400.0}
{'timestamp': 1649112588, 'cumulative_price': 4000.0}
{'timestamp': 1649112602, 'cumulative_price': 5600.0}
{'timestamp': 1649112616, 'cumulative_price': 7200.0}
{'timestamp': 1649112630, 'cumulative_price': 8800.0}
{'timestamp': 1649112644, 'cumulative_price': 10400.0}
{'timestamp': 1649112658, 'cumulative_price': 12000.0}
{'timestamp': 1649112672, 'cumulative_price': 13600.0}
1.0412500000000002
current_snapshot {'timestamp': 1649112672, 'cumulative_price': 1.0412500000000002}
{'timestamp': 1649112574, 'cumulative_price': 0.18375000000000002}
{'timestamp': 1649112588, 'cumulative_price': 0.30625}
{'timestamp': 1649112602, 'cumulative_price': 0.4287500000000001}
{'timestamp': 1649112616, 'cumulative_price': 0.5512500000000001}
{'timestamp': 1649112630, 'cumulative_price': 0.6737500000000002}
{'timestamp': 1649112644, 'cumulative_price': 0.796250000000000

TypeError: _ApplySwapToPool() missing 1 required positional argument: 'cur_timestamp'

In [307]:
cur_block_time = 1649100306

for i in range(0,100):    
    terra_classic_instance.astroport_provide_liquidity("trader", 100, 1, cur_block_time, 0 )
#     astroport_swap(self, trader, offer_asset, belief_price, max_spread)
    terra_classic_instance.record_twap_snapshots(cur_block_time)
    cur_block_time = cur_block_time + 121

# cur_block_time = cur_block_time + 121
# terra_classic_instance.astroport_provide_liquidity("trader", 100, 1, cur_block_time, 0 )
# terra_classic_instance.record_twap_snapshots(cur_block_time)

# cur_block_time = cur_block_time + 121
# terra_classic_instance.astroport_provide_liquidity("trader", 100, 1, cur_block_time, 0 )
# terra_classic_instance.record_twap_snapshots(cur_block_time)

# cur_block_time = cur_block_time + 121
# terra_classic_instance.astroport_provide_liquidity("trader", 100, 1, cur_block_time, 0 )
# terra_classic_instance.record_twap_snapshots(cur_block_time)

# cur_block_time = cur_block_time + 121
# terra_classic_instance.astroport_provide_liquidity("trader", 100, 1, cur_block_time, 0 )
# terra_classic_instance.record_twap_snapshots(cur_block_time)

# cur_block_time = cur_block_time + 121
# terra_classic_instance.astroport_provide_liquidity("trader", 100, 1, cur_block_time, 0 )
# terra_classic_instance.record_twap_snapshots(cur_block_time)

# cur_block_time = cur_block_time + 121
# terra_classic_instance.astroport_provide_liquidity("trader", 100, 1, cur_block_time, 0 )
# terra_classic_instance.record_twap_snapshots(cur_block_time)

# cur_block_time = cur_block_time + 121





ust
[]
Get cumulative price : ust
cumulative_price = 0 
latest snapshot = 0 
[{'timestamp': 1649100306, 'cumulative_price': 0}]

uluna
[]
Get cumulative price : uluna
cumulative_price = 0 
latest snapshot = 0 
[{'timestamp': 1649100306, 'cumulative_price': 0}]

ust
[{'timestamp': 1649100306, 'cumulative_price': 0}]
latest_snapshot = {'timestamp': 1649100306, 'cumulative_price': 0}
latest_snapshot = 121 | tolerance = 120
Get cumulative price : ust
cumulative_price = 0 
snapshot still in array = {'timestamp': 1649100306, 'cumulative_price': 0} 
latest snapshot = 0 
[{'timestamp': 1649100306, 'cumulative_price': 0}, {'timestamp': 1649100427, 'cumulative_price': 0}]

uluna
[{'timestamp': 1649100306, 'cumulative_price': 0}]
latest_snapshot = {'timestamp': 1649100306, 'cumulative_price': 0}
latest_snapshot = 121 | tolerance = 120
Get cumulative price : uluna
cumulative_price = 0 
snapshot still in array = {'timestamp': 1649100306, 'cumulative_price': 0} 
latest snapshot = 0 
[{'timestamp': 

In [309]:
terra_classic_instance.query_twap_price("ust",cur_block_time)

{'twap': 0.0, 'error': None}

In [310]:
terra_classic_instance.astroport_get_cumulative_price("ust")

0

In [130]:
# terra_classic_instance.BankKeeper.GetBalance("trader", "ust")
terra_classic_instance.BankKeeper.GetBalance("trader", "ust_uluna_lp_token")



24.06784757892085

In [139]:
terra_classic_instance.astroport_get_pool_info()

{'uluna_pool_size': 7,
 'ust_pool_size': 700,
 'lp_supply': 24.06784757892085,
 'block_time_last': 1649100348,
 'uluna_cumulative_last': 4200000000.0,
 'ust_cumulative_last': 420000.0}

In [133]:
terra_classic_instance.query_twap_price("ust",cur_block_time)

{'twap': 0, 'error': 'NoSnapshotWithinTolerance'}

In [134]:
terra_classic_instance.snapshots

{'ust': [{'timestamp': 1649100306, 'cumulative_price': 0}],
 'uluna': [{'timestamp': 1649100306, 'cumulative_price': 0}]}

In [55]:
crash_analysis_DF.head(3)

,BlockHeight,uusd::minted,uusd::burnt,uluna::minted,uluna::burnt,ExchangeRate::USD/LUNA,ExchangeRate::LUNA/UST,ExchangeRate::UST/LUNA,ExchangeRate::USD/UST,LUNA Supply,UST Supply,timestamp,dateTime
0,7117990,"10,434.7495680",0.0000000,0.0000000,94.2895450,110.7717770,0.0089910,111.2217760,0.9959540,"800,000,000.0000000","15,000,000,000.0000000",1649100306,2022-04-04 23:25:06
1,7117991,0.0000000,0.0000000,0.0000000,0.0000000,110.7781370,0.0089908,111.2253395,0.9959793,"800,000,000.0000000","15,000,000,000.0000000",1649100313,2022-04-04 23:25:13
2,7117992,0.0000000,0.0000000,0.0000000,0.0000000,110.7115950,0.0089908,111.2253395,0.9953810,"800,000,000.0000000","15,000,000,000.0000000",1649100320,2022-04-04 23:25:20
